## Homework 3 - Om Gaikwad

### Step 1: Data Acquisition

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import gzip, struct, os

# --- Step 1: Data Acquisition and Visualization ---
def read_idx(filename):
    with gzip.open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

X_train = read_idx("AAI-595 - Intro to Machine Learning\HW2\HW4\MNIST\t10k-images-idx3-ubyte.gz")
y_train = read_idx("HW4/MNIST/train-labels-idx1-ubyte.gz")
X_test = read_idx("HW4/MNIST/t10k-images-idx3-ubyte.gz")    
y_test = read_idx("HW4/MNIST/t10k-labels-idx1-ubyte.gz")

print("Train images shape:", X_train.shape)
print("Train labels shape:", y_train.shape)
print("Test images shape:", X_test.shape)
print("Test labels shape:", y_test.shape)

# visualize a sample
idx = np.random.randint(0, X_train.shape[0])
plt.imshow(X_train[idx], cmap='gray')
plt.title(f"Label: {y_train[idx]}")
plt.show()

<>:12: SyntaxWarning: invalid escape sequence '\H'
<>:12: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Om\AppData\Local\Temp\ipykernel_36640\79328142.py:12: SyntaxWarning: invalid escape sequence '\H'
  X_train = read_idx("AAI-595 - Intro to Machine Learning\HW2\HW4\MNIST\t10k-images-idx3-ubyte.gz")
C:\Users\Om\AppData\Local\Temp\ipykernel_36640\79328142.py:12: SyntaxWarning: invalid escape sequence '\H'
  X_train = read_idx("AAI-595 - Intro to Machine Learning\HW2\HW4\MNIST\t10k-images-idx3-ubyte.gz")


FileNotFoundError: [Errno 2] No such file or directory: 'AAI-595 - Intro to Machine Learning\\HW2\\HW4\\MNIST\t10k-images-idx3-ubyte.gz'

In [10]:
X_train = read_idx("HW4/MNIST/train-images-idx3-ubyte.gz")
y_train = read_idx("HW4/MNIST/train-labels-idx1-ubyte.gz")
X_test = read_idx("HW4/MNIST/t10k-images-idx3-ubyte.gz")    
y_test = read_idx("HW4/MNIST/t10k-labels-idx1-ubyte.gz")

print("Train images shape:", X_train.shape)
print("Train labels shape:", y_train.shape)
print("Test images shape:", X_test.shape)
print("Test labels shape:", y_test.shape)

# visualize a sample
idx = np.random.randint(0, X_train.shape[0])
plt.imshow(X_train[idx], cmap='gray')
plt.title(f"Label: {y_train[idx]}")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'HW4/MNIST/train-images-idx3-ubyte.gz'